In [2]:
ler_ibge_ipca_subitem <- function(data){
  library(readxl)
  stopifnot(is(data, "Date"), length(data) == 1)
  if(data >= "2020-01-01"){
    url <- format(
      data,
      "ftp://ftp.ibge.gov.br/Precos_Indices_de_Precos_ao_Consumidor/IPCA/Resultados_por_Subitem/ipca_%Y%mSubitem.zip")
  } else{
    url <- format(
      data,
      "ftp://ftp.ibge.gov.br/Precos_Indices_de_Precos_ao_Consumidor/IPCA/Resultados_por_Subitem/%Y/ipca_%Y%mSubitem.zip"
    )
  }
  filename <- format(data, "Gabriel André/Estudo R/ipca_%Y%mSubitem.zip")
  file <- download.file(url = url, destfile = filename, mode = "wb")
  files <- unzip(zipfile = filename, exdir = "Gabriel André/Estudo R")
  dados <- read_excel(path = files, sheet = 1, skip = 4, col_names = T)
  dados <- data.frame(dados)
  dados <- dados[-1,]
  file.remove(files)
  for(i in 2:ncol(dados)){
    dados[,i] <- suppressWarnings(as.numeric(dados[,i]))
  }
  return(View(dados))
  
}

In [1]:
grafico_ibge_ipca_subitem <- function(data, top_n){
  library(readxl)
  library(dplyr)
  library(tibble)
  library(ggplot2)
  library(ggthemes)
  stopifnot(is(data, "Date"), length(data) == 1, data >= as.Date("2017-10-01"))
  if(data >= "2020-01-01"){
    url <- format(
      data,
      "ftp://ftp.ibge.gov.br/Precos_Indices_de_Precos_ao_Consumidor/IPCA/Resultados_por_Subitem/ipca_%Y%mSubitem.zip")
  } else{
    url <- format(
      data,
      "ftp://ftp.ibge.gov.br/Precos_Indices_de_Precos_ao_Consumidor/IPCA/Resultados_por_Subitem/%Y/ipca_%Y%mSubitem.zip"
    )
  }
  filename <- format(data, "Gabriel André/Estudo R/ipca_%Y%mSubitem.zip")
  file <- download.file(url = url, destfile = filename, mode = "wb")
  files <- unzip(zipfile = filename, exdir = "Gabriel André/Estudo R")
  dados <- read_excel(path = files, sheet = 1, skip = 4, col_names = T)
  dados <- data.frame(dados)
  dados <- dados[-1,]
  file.remove(files)
  for(i in 2:ncol(dados)){
    dados[,i] <- suppressWarnings(as.numeric(dados[,i]))
  }
  dados %>%
    ungroup() %>%
    top_n(top_n, NACIONAL) %>%
    mutate(`...1` = reorder(`...1`, NACIONAL)) %>%
    ggplot()+
    geom_col(aes(x = `...1`, y = NACIONAL))+
    coord_flip()+
    labs(x = "", y = "%", title = if(top_n > 0){
      print("Produtos que mais subiram")
    } else{
      print("Produtos que mais caíram")
    },
         subtitle = format(data, "%B de %Y"), caption = "Fonte:IBGE")+
    theme_economist()
}
